#Gensim 내장 LDA 알고리즘 사용

In [ ]:
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git

In [ ]:
cd Mecab-ko-for-Google-Colab

In [ ]:
!bash install_mecab-ko_on_colab190912.sh

In [ ]:
cd /content/mecab-ko-dic-2.1.1-20180720

In [ ]:
ls user-dic

In [6]:
cp "/content/drive/My Drive/Colab Notebooks/output/nnp.csv" "./user-dic/nnp.csv"

In [ ]:
# 정상적으로 DRIVE에서 COPY 되었는지 확인
with open("./user-dic/nnp.csv", 'r', encoding='utf-8') as f: 
    file_new = f.readlines() 
file_new

In [ ]:
ls tools

In [ ]:
!bash ./tools/add-userdic.sh

In [ ]:
!make install

In [ ]:
from konlpy.tag import Mecab
from collections import Counter

mecab = Mecab()
nouns = mecab.nouns(ARTICLE)

In [13]:
import pandas as pd
from tqdm import tqdm
from collections import Counter

In [19]:
cd "/content/drive/My Drive/Colab Notebooks"

/content/drive/My Drive/Colab Notebooks


In [21]:
data1 = "data/MedicalDaily_crawling_edit.csv"
df = pd.read_csv(data1, names=['text'], dtype={'text': str})

In [40]:
word_list = []
for idx in tqdm(range(len(df))):
    try:
        nouns = mecab.nouns(df.loc[idx, 'text'])
        word_list.append(nouns)
    except Exception as e:
        countinue

100%|██████████| 989/989 [00:04<00:00, 243.05it/s]


In [ ]:
word_list[0:2]

In [42]:
import gensim
from gensim import corpora, models
from gensim.models import CoherenceModel

In [72]:
# create Dictionary
id2word=corpora.Dictionary(word_list)
id2word.filter_extremes(no_below=20)

# create Corpus
texts = word_list

# Term Document Frequency
corpus=[id2word.doc2bow(text) for text in texts]

In [ ]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

In [ ]:
pip install pyLDAvis

In [80]:
from pprint import pprint
 
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
 
# spacy for lemmatization
import spacy
 
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline
 
 
# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)
 
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [81]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=id2word, num_topics=20, 
                                            random_state=100, update_every=1, chunksize=100, 
                                            passes=10, alpha='auto', per_word_topics=True)

In [ ]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

In [84]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus)) # a measure of how good the model is. lower the better.
 
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=texts, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -6.282816343051974

Coherence Score:  0.506610251966864


In [85]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
17     0.176588  0.058554       1        1  13.042140
16     0.265990  0.148261       2        1   8.873618
4     -0.069318  0.166820       3        1   8.715932
19    -0.092113 -0.081132       4        1   6.424389
13    -0.154214  0.135728       5        1   5.698703
14     0.035545 -0.164660       6        1   5.597962
9     -0.068562 -0.114132       7        1   5.597480
15    -0.054138 -0.143883       8        1   5.491891
18     0.279675  0.027517       9        1   5.192608
3     -0.114908  0.011296      10        1   5.122914
6     -0.182324  0.027225      11        1   4.957963
1      0.172545  0.143176      12        1   4.358915
7     -0.122648 -0.001797      13        1   4.281873
5     -0.155976  0.122919      14        1   3.992319
8     -0.101413 -0.074412      15        1   3.052032
12    -0.046285  0.122835      16        1   2.633750
0     -0.121718  0.070243      17        1   2.326194
2      0.073290 -0.264871      18        1   1.915458
11     0.217007 -0.023728      19        1   1.484370
10     0.062977 -0.165959      20        1   1.239489, topic_info=    Term         Freq        Total Category  logprob  loglift
396   수술  2550.000000  2550.000000  Default  30.0000  30.0000
398    암  1596.000000  1596.000000  Default  29.0000  29.0000
34    의사  1026.000000  1026.000000  Default  28.0000  28.0000
229   의료  2614.000000  2614.000000  Default  27.0000  27.0000
461   검사   930.000000   930.000000  Default  26.0000  26.0000
..   ...          ...          ...      ...      ...      ...
344   원장    19.977190   214.562254  Topic20  -4.7051   2.0165
14    논문    19.735961   214.793024  Topic20  -4.7172   2.0032
409   정상    22.687646   344.206773  Topic20  -4.5778   1.6710
247   진행    23.244150   688.239294  Topic20  -4.5536   1.0024
49    최근    20.880398   523.806739  Topic20  -4.6608   1.1682

[1000 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
0         1  0.038232    가능
0         2  0.044398    가능
0         3  0.080163    가능
0         4  0.129495    가능
0         5  0.006166    가능
...     ...       ...   ...
1054      6  0.090238  흉부외과
1054      7  0.794095  흉부외과
1123      4  0.980604    흉터
928      13  0.992712    흡연
1148      5  0.992686   흡연자

[2739 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[18, 17, 5, 20, 14, 15, 10, 16, 19, 4, 7, 2, 8, 6, 9, 13, 1, 3, 12, 11])